# Testing the Model Out on New Data 

This is a demo to test out the model on new data. The main project table for training was created using relevant data exports from Jan 1, 2018 to May 31, 2022. The team wrangled new data from April 1, 2022 to June 8, 2022 to identical format the data the model was trained on. 

This notebook is taking the new data and running it through the team's saved higher performing model, the ExtraTreesClassifier, and investigating the prediction results on new comepletely unseen data from a more recent date range. 

Although the recall was very low on this new data attempt, it was not 0. As shown at the end of the notebook, the following wildfires were predicted using the team's model. 

### Results

These are actual named WFIGS Incidents which were predicted using the teams model:

- BK Inman Creek A
- Beaver River Fire
- Sand Creek
- WILDFLOWER
- County Road 56
- Clear Creek
- Persimmon Gully
- HOMOCHITTO BB25
- LAC-122039
- Sheridan Lake
- Wolf Swamp
- C-24 28
- BIGGS
- MIDDLE MARCH

From doing some googling, below are links of several of these events. It seems like the dates in the articles and reports align with the time range of the wrangled data. The model was able to identify these wildfire starts on the day the fires started.

Beaver River Fire: https://abc7amarillo.com/news/local/beave-river-fire-2-firefighters-injured-2-homes-destroyed-24500-acres-burned 

MIDDLE MARCH: https://www.youtube.com/watch?v=krl5LZAIFw8 

County Road 56: https://thenwfireblog.com/2022/04/12/co-wildfire-county-road-56-fire-1/ 


Persimmon Gully: https://www.kplctv.com/2022/04/08/bud-bennett-fire-remains-within-containment-lines/ 


C-24 28: https://inciweb.nwcg.gov/incident/8069/ 

BIGGS: https://www.fireweatheravalanche.org/wildfire/incident/247104/new-mexico/biggs-fire 

In [1]:
import pandas as pd
import geopy.distance  
import numpy as np
pd.set_option('display.max_columns', 200)

In [2]:
#pull in the new unseen data into a FIRMS and SCAN merged dataframe from April 1, 2022 to June 8, 2022
df_FirmsAndScan = pd.read_csv('April1ToJune8FullTable.csv')
df_FirmsAndScan.head()

C:\Users\anderb4\AppData\Local\Temp\ipykernel_113272\1170095749.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_FirmsAndScan = pd.read_csv('April1ToJune8FullTable.csv')


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),date_new,nearbydetections
0,25.97230,-80.52075,304.22,1.04,1.02,2022-04-01,332,Terra,MODIS,27,6.1NRT,292.23,4.15,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.909398,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0
1,25.98871,-80.52980,313.08,1.04,1.02,2022-04-01,332,Terra,MODIS,86,6.1NRT,292.95,11.20,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.371108,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0
2,25.98096,-80.51802,310.56,1.04,1.02,2022-04-01,332,Terra,MODIS,80,6.1NRT,292.60,9.16,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.538551,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0
3,25.94230,-80.53675,305.96,1.04,1.02,2022-04-01,332,Terra,MODIS,66,6.1NRT,292.65,6.19,N,25.9,-80.5,3.0,"2022-04-01,25.9,-80.5","2022-04-01,03,25.9,-80.5",2022-04-01,Mana House,41.453970,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0
4,25.96955,-80.54127,320.76,1.04,1.02,2022-04-01,332,Terra,MODIS,100,6.1NRT,293.00,17.93,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.289368,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0


In [3]:
#pull in the wfigs data
df_wfigsbig = pd.read_csv('WFIGS_PulledJune82022.csv')
df_wfigsbig.head()

C:\Users\anderb4\AppData\Local\Temp\ipykernel_113272\2160030664.py:1: DtypeWarning: Columns (13,14,32,61,64,67,68,69,70,72,77,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wfigsbig = pd.read_csv('WFIGS_PulledJune82022.csv')


,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID
0,-104.457511,45.785037,1,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.6,20.0,MTMCC,NaN,NaN,NaN,2020/08/12 20:45:59+00,NaN,NaN,NaN,NaN,Human,Equipment,Farming Equipment,NDU0,NaN,2020/08/06 18:58:00+00,NaN,2020/08/12 14:00:00+00,NaN,NaN,NaN,NaN,PN,1502,NRCC,NaN,NaN,NaN,NaN,NaN,Ramme,NaN,WF,FI,45.78496,-104.4958,NaN,NaN,{FE67856C-7401-44BD-B6F3-9A57EF990079},NaN,0,1.0,0.0,0.0,0.0,NaN,000224,NaN,NaN,NaN,Carter,MTMCC,30011.0,C&L,MTLG42,NaN,Private,Private,NaN,NaN,NaN,NaN,NaN,NaN,NR16,C&L,MTLG42,US-MT,Grass,GR4,NaN,NaN,NaN,2020-MTLG42-000224,No Decision,wildcad,INFORM_Inspector,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN
1,-118.180712,33.808985,2,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.1,CALACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,2020/02/28 20:45:40+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,LAC-066100,NaN,WF,FI,33.80898,-118.1807,NaN,NaN,{6A311ABB-DF4F-4947-B8DD-3900BDA784F6},NaN,0,NaN,NaN,NaN,NaN,NaN,066100,NaN,NaN,NaN,Los Angeles,CALACC,6037.0,NaN,CACAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SC08,C&L,CALAC,US-CA,NaN,NaN,NaN,NaN,NaN,2020-CALAC-066100,No Decision,lacocad,lacocad,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN
2,-114.835414,48.073946,3,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.0,50.0,MTKIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,NaN,NaN,NaN,NaN,2017/10/17 20:20:24+00,NaN,2017/11/09 21:59:59+00,NaN,NaN,NaN,NaN,NaN,NaN,NRCC,NaN,NaN,NaN,NaN,NaN,South Lost Prairie,NaN,WF,FI,48.07167,-114.8303,1.0,NaN,{9599AE8E-66B8-4412-B849-7271271B0463},NaN,0,0.0,0.0,0.0,0.0,NaN,000878,NaN,NaN,NaN,Flathead,NaN,30029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR02,DNRC,MTNWS,US-MT,NaN,NaN,NaN,NaN,NaN,2017-MTNWS-000878,NaN,wildcad,wildcad,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN
3,-117.153901,33.176394,4,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,CAMVIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MC2S,NaN,2019/07/01 19:54:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,SYCAMORE,NaN,WF,FI,NaN,NaN,NaN,NaN,{1AF2C949-B159-4

In [4]:
# now starting to transform the fire db
latlonground = 1 #Sets the number of decimal places to round latlong
#getting the fire discovered date into the same format as the same date
df_wfigsbig['disc_date'] = df_wfigsbig['FireDiscoveryDateTime'].astype(str).str[0:10]
df_wfigsbig['disc_date'] = df_wfigsbig['disc_date'].astype(str).str.replace('/', '-')
# #convert the init long and lat frame to 2 decimals like i did in the satellite df 
df_wfigsbig['init_lat_rounded'] = round(df_wfigsbig['Y'], latlonground) #rounds lat long into new column, using latlonground # of decimal pts
df_wfigsbig['init_long_rounded'] = round(df_wfigsbig['X'], latlonground) #rounds lat long into new column, using latlonground # of decimal pts
#make the date loc column
df_wfigsbig['disc_date_loc'] = df_wfigsbig['disc_date'] + ',' + df_wfigsbig['init_lat_rounded'].astype(str) + ',' + df_wfigsbig['init_long_rounded'].astype(str)
df_wfigsbig.head()

,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc
0,-104.457511,45.785037,1,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.6,20.0,MTMCC,NaN,NaN,NaN,2020/08/12 20:45:59+00,NaN,NaN,NaN,NaN,Human,Equipment,Farming Equipment,NDU0,NaN,2020/08/06 18:58:00+00,NaN,2020/08/12 14:00:00+00,NaN,NaN,NaN,NaN,PN,1502,NRCC,NaN,NaN,NaN,NaN,NaN,Ramme,NaN,WF,FI,45.78496,-104.4958,NaN,NaN,{FE67856C-7401-44BD-B6F3-9A57EF990079},NaN,0,1.0,0.0,0.0,0.0,NaN,000224,NaN,NaN,NaN,Carter,MTMCC,30011.0,C&L,MTLG42,NaN,Private,Private,NaN,NaN,NaN,NaN,NaN,NaN,NR16,C&L,MTLG42,US-MT,Grass,GR4,NaN,NaN,NaN,2020-MTLG42-000224,No Decision,wildcad,INFORM_Inspector,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN,2020-08-06,45.8,-104.5,"2020-08-06,45.8,-104.5"
1,-118.180712,33.808985,2,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.1,CALACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,2020/02/28 20:45:40+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,LAC-066100,NaN,WF,FI,33.80898,-118.1807,NaN,NaN,{6A311ABB-DF4F-4947-B8DD-3900BDA784F6},NaN,0,NaN,NaN,NaN,NaN,NaN,066100,NaN,NaN,NaN,Los Angeles,CALACC,6037.0,NaN,CACAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SC08,C&L,CALAC,US-CA,NaN,NaN,NaN,NaN,NaN,2020-CALAC-066100,No Decision,lacocad,lacocad,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN,2020-02-28,33.8,-118.2,"2020-02-28,33.8,-118.2"
2,-114.835414,48.073946,3,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.0,50.0,MTKIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,NaN,NaN,NaN,NaN,2017/10/17 20:20:24+00,NaN,2017/11/09 21:59:59+00,NaN,NaN,NaN,NaN,NaN,NaN,NRCC,NaN,NaN,NaN,NaN,NaN,South Lost Prairie,NaN,WF,FI,48.07167,-114.8303,1.0,NaN,{9599AE8E-66B8-4412-B849-7271271B0463},NaN,0,0.0,0.0,0.0,0.0,NaN,000878,NaN,NaN,NaN,Flathead,NaN,30029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR02,DNRC,MTNWS,US-MT,NaN,NaN,NaN,NaN,NaN,2017-MTNWS-000878,NaN,wildcad,wildcad,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN,2017-10-17,48.1,-114.8,"2017-10-17,48.1,-114.8"
3,-117.153901,33.176394,4,NaN,DEFAULT,NaN,NaN,

In [5]:
#Merging on the date + location key.
dfDiscovered = df_FirmsAndScan.merge(df_wfigsbig, left_on='date_loc', right_on='disc_date_loc', how='inner') #creates df of only dsetected areas
dfMerged = df_FirmsAndScan.merge(df_wfigsbig, left_on='date_loc', right_on='disc_date_loc', how='left') #creates df of both 
dfMerged['FIRE_DETECTED'] = dfMerged['OBJECTID'].isnull() #if there was no fire detectection set to true
dfMerged['FIRE_DETECTED'] = ~dfMerged['FIRE_DETECTED'].astype(bool) #flip so that if detection merged set to true
dfFULL = dfMerged #use this for matching trues later 
print(dfMerged.shape)
dfMerged.head()

(232618, 146)


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),date_new,nearbydetections,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc,FIRE_DETECTED
0,25.97230,-80.52075,304.22,1.04,1.02,2022-04-01,332,Terra,MODIS,27,6.1NRT,292.23,4.15,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.909398,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,25.98871,-80.52980,313.08,1.04,1.02,2022-04-01,332,Terra,MODIS,86,6.1NRT,292.95,11.20,N,26.0,-80.5,3.0,"2022-04-01,26.0,-80.5","2022-04-01,03,26.0,-80.5",2022-04-01,Mana House,41.371108,2022-04-01,2102.0,HI,SCAN,Mana House,4234.0,19.95658,-155.53517,Hawaii,Hawaii Region,Hawaii,Hawaii,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,2022-04-01,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
#check the number of merges
print(dfDiscovered.shape)
dfDiscovered.head(10)

(7020, 145)


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),date_new,nearbydetections,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc
0,33.04139,-79.71236,310.91,1.70,1.28,2022-04-01,1539,Terra,MODIS,70,6.1NRT,293.24,17.77,D,33.0,-79.7,15.0,"2022-04-01,33.0,-79.7","2022-04-01,15,33.0,-79.7",2022-04-01,N Piedmont Arec,86.107262,2022-04-01,2039.0,VA,SCAN,N Piedmont Arec,520.0,38.23333,-78.11667,Orange,Mid Atlantic Region,Lower Chesapeake,Lower Chesapeake,Pamunkey,13.2,0.05,4.52,52.0,38.2,35.9,60.0,8.8,2022-04-01,11.0,-79.710283,33.041395,247716,NaN,FIREREPORTING,117.7206,2022/04/07 19:15:00+00,2022/04/16 21:30:00+00,200.0,100.0,SCSCC,55000.0,NaN,NaN,NaN,Minimal,Smoldering,Creeping,NaN,Undetermined,NaN,NaN,PJP2,NaN,2022/04/01 22:04:00+00,Type 4 Incident,2022/04/21 17:18:00+00,0.0,100.0,0.0,0.0,P8,812.0,SACC,2022/04/07 18:59:59+00,2022/04/04 14:00:00+00,2022/04/07 18:59:59+00,F,Type 4 IC,205,20 miles north of Mt Pleasant SC,WF,FI,33.041310,-79.710140,250.0,NaN,{4F3AF3FA-F807-4923-8B51-0CEB54E8463D},NaN,0,1.0,0.0,0.0,0.0,0.0,000166,99.0,100.0,NaN,Charleston,SCSCC,45019,USFS,SCFMF,NaN,USFS,Federal,NaN,NaN,NaN,NaN,NaN,NaN,SA37,USFS,SCFMF,US-SC,NaN,NaN,Timber (Grass and Understory),NaN,4.0,2022-SCFMF-000166,Approved Decision,wildcad,wildcad,0,Type 4 Incident,2022/04/04 22:31:34+00,2022/04/01 23:08:12+00,2022/04/21 17:19:12+00,IRWIN,{B0DD1059-0A99-4E2E-A7DB-9A82BE515C3D},0.0,NaN,NaN,2022-04-01,33.0,-79.7,"2022-04-01,33.0,-79.7"
1,33.04439,-79.71879,303.87,1.70,1.28,2022-04-01,1539,Terra,MODIS,54,6.1NRT,292.98,8.78,D,33.0,-79.7,15.0,"2022-04-01,33.0,-79.7","2022-04-01,15,33.0,-79.7",2022-04-01,

## Clean and Prep the Data

Clean and prepare the data to match the dataframe the saved model is expecting 

In [7]:
#beign cleaning the data by removing unnecessary columns:
#drop the columns we VERY likely do not need for this approach
coltodrop = [
'acq_date',
'acq_time',
'version',
'lat',
'long',
'hour',
'date_loc',
'date_hour_loc',
'newdate',
'nearestStation',
'Date',
'Station Id',
'State Code',
'Network Code',
'Station Name',
'Elevation (ft)',
'Latitude',
'Longitude',
'County Name',
'HUC6 Name',
'HUC8 Name',
'date_new',
'X',
'Y',
'OBJECTID',
'ABCDMisc',
'ADSPermissionState',
'CalculatedAcres',
'ContainmentDateTime',
'ControlDateTime',
'DailyAcres',
'DiscoveryAcres',
'DispatchCenterID',
'EstimatedCostToDate',
'FinalFireReportApprovedByTitle',
'FinalFireReportApprovedByUnit',
'FinalFireReportApprovedDate',
'FireBehaviorGeneral',
'FireBehaviorGeneral1',
'FireBehaviorGeneral2',
'FireBehaviorGeneral3',
'FireCause',
'FireCauseGeneral',
'FireCauseSpecific',
'FireCode',
'FireDepartmentID',
'FireDiscoveryDateTime',
'FireMgmtComplexity',
'FireOutDateTime',
'FireStrategyConfinePercent',
'FireStrategyFullSuppPercent',
'FireStrategyMonitorPercent',
'FireStrategyPointZonePercent',
'FSJobCode',
'FSOverrideCode',
'GACC',
'ICS209ReportDateTime',
'ICS209ReportForTimePeriodFrom',
'ICS209ReportForTimePeriodTo',
'ICS209ReportStatus',
'IncidentManagementOrganization',
'IncidentName',
'IncidentShortDescription',
'IncidentTypeCategory',
'IncidentTypeKind',
'InitialLatitude',
'InitialLongitude',
'InitialResponseAcres',
'InitialResponseDateTime',
'IrwinID',
'IsFireCauseInvestigated',
'IsFireCodeRequested',
'IsFSAssisted',
'IsMultiJurisdictional',
'IsReimbursable',
'IsTrespass',
'IsUnifiedCommand',
'LocalIncidentIdentifier',
'PercentContained',
'PercentPerimeterToBeContained',
'POOCity',
'POOCounty',
'POODispatchCenterID',
'POOFips',
'POOJurisdictionalAgency',
'POOJurisdictionalUnit',
'POOJurisdictionalUnitParentUnit',
'POOLandownerCategory',
'POOLandownerKind',
'POOLegalDescPrincipalMeridian',
'POOLegalDescQtr',
'POOLegalDescQtrQtr',
'POOLegalDescRange',
'POOLegalDescSection',
'POOLegalDescTownship',
'POOPredictiveServiceAreaID',
'POOProtectingAgency',
'POOProtectingUnit',
'POOState',
'PredominantFuelGroup',
'PredominantFuelModel',
'PrimaryFuelModel',
'SecondaryFuelModel',
'TotalIncidentPersonnel',
'UniqueFireIdentifier',
'WFDSSDecisionStatus',
'CreatedBySystem',
'ModifiedBySystem',
'IsDispatchComplete',
'OrganizationalAssessment',
'StrategicDecisionPublishDate',
'CreatedOnDateTime_dt',
'ModifiedOnDateTime_dt',
'Source',
'GlobalID',
'IsCpxChild',
'CpxName',
'CpxID',
'disc_date',
'init_lat_rounded',
'init_long_rounded',
'disc_date_loc'
            ]
dfMerged = dfMerged.drop(coltodrop, axis = 1) #drops unwanted columns for ML process
dfMerged.head()

,latitude,longitude,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,daynight,StationDist,HUC2 Name,HUC4 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,25.97230,-80.52075,304.22,1.04,1.02,Terra,MODIS,27,292.23,4.15,N,41.909398,Hawaii Region,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,6.0,False
1,25.98871,-80.52980,313.08,1.04,1.02,Terra,MODIS,86,292.95,11.20,N,41.371108,Hawaii Region,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,6.0,False
2,25.98096,-80.51802,310.56,1.04,1.02,Terra,MODIS,80,292.60,9.16,N,41.538551,Hawaii Region,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,6.0,False
3,25.94230,-80.53675,305.96,1.04,1.02,Terra,MODIS,66,292.65,6.19,N,41.453970,Hawaii Region,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,6.0,False
4,25.96955,-80.54127,320.76,1.04,1.02,Terra,MODIS,100,293.00,17.93,N,41.289368,Hawaii Region,Hawaii,14.2,0.06,NaN,58.0,15.3,21.4,84.0,7.6,6.0,False


In [8]:
#remove more columns we dont need for table creation 
coltodrop = [
    'latitude',
    'longitude',
    'daynight',
    'Soil Moisture Percent -4in (pct) Start of Day Values',
    'HUC4 Name',
    'Precipitation Month-to-date (in) Start of Day Values',
    'StationDist'
            ]
dfMerged = dfMerged.drop(coltodrop, axis = 1) #drops unwanted columns for ML process
dfMerged.head()

,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,304.22,1.04,1.02,Terra,MODIS,27,292.23,4.15,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
1,313.08,1.04,1.02,Terra,MODIS,86,292.95,11.20,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
2,310.56,1.04,1.02,Terra,MODIS,80,292.60,9.16,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
3,305.96,1.04,1.02,Terra,MODIS,66,292.65,6.19,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
4,320.76,1.04,1.02,Terra,MODIS,100,293.00,17.93,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False


In [9]:
#checking some counts of values on missing columns 
collist = list(dfMerged.columns) #get the list of columns in the df
numbrows = dfMerged.shape[0] #saves number of riows in the df
for col in collist:
    missingcount = dfMerged[col].isnull().sum() #count missing vals 
    missingpercent = round((missingcount/numbrows)*100, 2) #gets the percent compared to the entire dataframe
    print('number of nan for '+col+':'+str(missingcount)+' ; '+str(missingpercent)+'%') #prints the result

number of nan for brightness:0 ; 0.0%
number of nan for scan:0 ; 0.0%
number of nan for track:0 ; 0.0%
number of nan for satellite:0 ; 0.0%
number of nan for instrument:0 ; 0.0%
number of nan for confidence:0 ; 0.0%
number of nan for bright_t31:0 ; 0.0%
number of nan for frp:0 ; 0.0%
number of nan for HUC2 Name:3536 ; 1.52%
number of nan for Precipitation Accumulation (in) Start of Day Values:39120 ; 16.82%
number of nan for Precipitation Increment (in):44360 ; 19.07%
number of nan for Air Temperature Average (degF):11560 ; 4.97%
number of nan for Soil Moisture Percent -2in (pct) Start of Day Values:31726 ; 13.64%
number of nan for Relative Humidity Enclosure (pct):11037 ; 4.74%
number of nan for Wind Speed Average (mph):13752 ; 5.91%
number of nan for nearbydetections:0 ; 0.0%
number of nan for FIRE_DETECTED:0 ; 0.0%


In [10]:
#now drop the nan rows 
print('df shape before dropping all nan: '+str(dfMerged.shape))
dfMerged = dfMerged.dropna() #drop all nan values 
print('df shape after dropping all nan: '+str(dfMerged.shape))

df shape before dropping all nan: (232618, 17)
df shape after dropping all nan: (160821, 17)


In [11]:
#check how many trues we have
dfMerged['FIRE_DETECTED'].value_counts()

False    155149
True       5672
Name: FIRE_DETECTED, dtype: int64

In [12]:
dfMerged.head()

,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,304.22,1.04,1.02,Terra,MODIS,27,292.23,4.15,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
1,313.08,1.04,1.02,Terra,MODIS,86,292.95,11.20,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
2,310.56,1.04,1.02,Terra,MODIS,80,292.60,9.16,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
3,305.96,1.04,1.02,Terra,MODIS,66,292.65,6.19,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False
4,320.76,1.04,1.02,Terra,MODIS,100,293.00,17.93,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False


In [13]:
dfMerged['satellite'].unique().tolist()

['Terra', 'Aqua', '1', 1, 'N']

In [14]:
#looks like some 1's are strings and some are ints. converting everything to string
dfMerged['satellite'] = dfMerged['satellite'].astype(str)
#check to make sure we are good now
dfMerged['satellite'].value_counts()

N        72916
1        72187
Terra     8902
Aqua      6816
Name: satellite, dtype: int64

In [15]:
#add the one hot encoded df to the merged dataframe and drop the original satellite column
sat_dummies = pd.get_dummies(dfMerged.satellite)
dfMerged = pd.concat([dfMerged, sat_dummies], axis=1) #adds in the one hot encoded  staellite df
dfMerged = dfMerged.drop('satellite', axis = 1) #drop the satellite column
dfMerged.head()

,brightness,scan,track,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,N,Terra
0,304.22,1.04,1.02,MODIS,27,292.23,4.15,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,0,1
1,313.08,1.04,1.02,MODIS,86,292.95,11.20,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,0,1
2,310.56,1.04,1.02,MODIS,80,292.60,9.16,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,0,1
3,305.96,1.04,1.02,MODIS,66,292.65,6.19,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,0,1
4,320.76,1.04,1.02,MODIS,100,293.00,17.93,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,0,1


In [16]:
#'N' was not part of the orginal model, need to drop
dfMerged=dfMerged.drop(['N'], axis=1)
dfMerged.head()

,brightness,scan,track,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra
0,304.22,1.04,1.02,MODIS,27,292.23,4.15,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1
1,313.08,1.04,1.02,MODIS,86,292.95,11.20,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1
2,310.56,1.04,1.02,MODIS,80,292.60,9.16,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1
3,305.96,1.04,1.02,MODIS,66,292.65,6.19,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1
4,320.76,1.04,1.02,MODIS,100,293.00,17.93,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1


In [17]:
#now one hot encode instrument
ohe = pd.get_dummies(dfMerged.instrument) #creates the one hot encoded df of instrument
dfMerged = pd.concat([dfMerged, ohe], axis=1) #merges the dataframe with the one hot encoded df
dfMerged = dfMerged.drop('instrument', axis = 1) #drop the instrument column
dfMerged.head()

,brightness,scan,track,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra,MODIS,VIIRS
0,304.22,1.04,1.02,27,292.23,4.15,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0
1,313.08,1.04,1.02,86,292.95,11.20,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0
2,310.56,1.04,1.02,80,292.60,9.16,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0
3,305.96,1.04,1.02,66,292.65,6.19,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0
4,320.76,1.04,1.02,100,293.00,17.93,Hawaii Region,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0


In [18]:
#because there is only 2 instruments (MODIS and VIIRS) we can drop one of the columns 
#we will keep the MODIS column, so 1 = MODIS and 0 = VIIRS instrument
dfMerged = dfMerged.drop('VIIRS', axis = 1) #drop the instrument column

In [19]:
#the column name 'HUC2 Name' will not work for the necessary syntax because it has a space in it. 
dfMerged = dfMerged.rename(columns={"HUC2 Name": "HUC2Name"}) #renames the HUC2 Name column to remove spaces 
#now one hot encode the region by HUC2 name:
ohe = pd.get_dummies(dfMerged.HUC2Name) #creates the one hot encoded df of instrument
dfMerged = pd.concat([dfMerged, ohe], axis=1) #merges the dataframe with the one hot encoded df
dfMerged = dfMerged.drop('HUC2Name', axis = 1) #drop the instrument column
dfMerged.head()

,brightness,scan,track,confidence,bright_t31,frp,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra,MODIS,Arkansas-White-Red Region,California Region,Great Basin Region,Great Lakes Region,Hawaii Region,Lower Colorado Region,Lower Mississippi Region,Mid Atlantic Region,Missouri Region,New England Region,Ohio Region,Pacific Northwest Region,Rio Grande Region,South Atlantic-Gulf Region,Tennessee Region,Texas-Gulf Region,Upper Colorado Region,Upper Mississippi Region
0,304.22,1.04,1.02,27,292.23,4.15,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,313.08,1.04,1.02,86,292.95,11.20,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,310.56,1.04,1.02,80,292.60,9.16,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,305.96,1.04,1.02,66,292.65,6.19,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,320.76,1.04,1.02,100,293.00,17.93,14.2,0.06,58.0,15.3,84.0,7.6,6.0,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
#'confidence' appears to be starnge. based off the link below: n=nominal l = low and h = high
# https://cdn.earthdata.nasa.gov/conduit/upload/10575/MODIS_C6_Fire_User_Guide_B.pdf
#another helpful link:
# https://earthdata.nasa.gov/faq/firms-faq#ed-confidence
# lets approximate low 20, nominal 50, and high 80 for this table 
# this can be adjusted later, if needed
mapping = {'l': 20, 'n': 50, 'h': 100}
dfMerged = dfMerged.replace({'confidence': mapping})
dfMerged['confidence'] = pd.to_numeric(dfMerged['confidence']) #convert confidence from string to integer

In [21]:
#make sure that all regions are in the df
listofregions = ['Arkansas-White-Red Region', 'California Region', 'Great Basin Region',
'Great Lakes Region', 'Hawaii Region', 'Lower Colorado Region',
'Lower Mississippi Region', 'Mid Atlantic Region', 'Missouri Region',
'New England Region', 'Ohio Region', 'Pacific Northwest Region',
'Rio Grande Region', 'Souris-Red-Rainy Region',
'South Atlantic-Gulf Region', 'Tennessee Region', 'Texas-Gulf Region',
'Upper Colorado Region', 'Upper Mississippi Region']
for region in listofregions:
    if region not in dfMerged:
        dfMerged[region] = 0 #create the region and mark as false if there were none 

In [22]:
#rearrange the order to match the MLtable
colstomatch = ['brightness', 'scan', 'track', 'confidence', 'bright_t31',
       'frp', 'Precipitation Accumulation (in) Start of Day Values',
       'Precipitation Increment (in)', 'Air Temperature Average (degF)',
       'Soil Moisture Percent -2in (pct) Start of Day Values',
       'Relative Humidity Enclosure (pct)', 'Wind Speed Average (mph)',
       'nearbydetections', 'FIRE_DETECTED', '1', 'Aqua', 'Terra', 'MODIS',
       'Arkansas-White-Red Region', 'California Region', 'Great Basin Region',
       'Great Lakes Region', 'Hawaii Region', 'Lower Colorado Region',
       'Lower Mississippi Region', 'Mid Atlantic Region', 'Missouri Region',
       'New England Region', 'Ohio Region', 'Pacific Northwest Region',
       'Rio Grande Region', 'Souris-Red-Rainy Region',
       'South Atlantic-Gulf Region', 'Tennessee Region', 'Texas-Gulf Region',
       'Upper Colorado Region', 'Upper Mississippi Region']
dfMerged=dfMerged[colstomatch]

In [23]:
#check the data types to make sure they are all values and not strings
dfMerged.dtypes

brightness                                              float64
scan                                                    float64
track                                                   float64
confidence                                                int64
bright_t31                                              float64
frp                                                     float64
Precipitation Accumulation (in) Start of Day Values     float64
Precipitation Increment (in)                            float64
Air Temperature Average (degF)                          float64
Soil Moisture Percent -2in (pct) Start of Day Values    float64
Relative Humidity Enclosure (pct)                       float64
Wind Speed Average (mph)                                float64
nearbydetections                                        float64
FIRE_DETECTED                                              bool
1                                                         uint8
Aqua                                    

DF should be ready for ML

## Machine Learning Prep

In [24]:
import pandas as pd
import pickle
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
#seperate data sets as labels and features
X = dfMerged.drop('FIRE_DETECTED', axis=1)
y = dfMerged['FIRE_DETECTED']

In [26]:
#load in the model
filename=r'C:\Users\anderb4\Documents\GeorgetownDSCert\WildfireCapstone\LiveDemo\LiveDemoPrep_InitialApproach\extratreesmodel.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [27]:
pred = loaded_model.predict(X) #predict the fires
pred[:20] #take a look at top predictions 

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [28]:
#see how the model peformed
print('Report:')
print(classification_report(y, pred))
print('Confusion matrix:')
print(confusion_matrix(y, pred))

Report:
              precision    recall  f1-score   support

       False       0.97      1.00      0.98    155149
        True       0.67      0.01      0.02      5672

    accuracy                           0.96    160821
   macro avg       0.82      0.50      0.50    160821
weighted avg       0.95      0.96      0.95    160821

Confusion matrix:
[[155125     24]
 [  5624     48]]


Not great results compared to the training, but the recall was not 0. Now to check which wildfires were accuratley predicted.

## Investigate predictions

Reverse engineer the machine learning dataframe to see what the accurate predictions were

In [29]:
#get a series of whether p[rediction was right or not
result = y==pred

In [30]:
#create a df of the result 
dfResult=pd.concat([X, y, result], axis=1)
dfResult.columns.values[37] = "PredResult"
dfResult.head()

,brightness,scan,track,confidence,bright_t31,frp,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,1,Aqua,Terra,MODIS,Arkansas-White-Red Region,California Region,Great Basin Region,Great Lakes Region,Hawaii Region,Lower Colorado Region,Lower Mississippi Region,Mid Atlantic Region,Missouri Region,New England Region,Ohio Region,Pacific Northwest Region,Rio Grande Region,Souris-Red-Rainy Region,South Atlantic-Gulf Region,Tennessee Region,Texas-Gulf Region,Upper Colorado Region,Upper Mississippi Region,FIRE_DETECTED,PredResult
0,304.22,1.04,1.02,27,292.23,4.15,14.2,0.06,58.0,15.3,84.0,7.6,6.0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
1,313.08,1.04,1.02,86,292.95,11.20,14.2,0.06,58.0,15.3,84.0,7.6,6.0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
2,310.56,1.04,1.02,80,292.60,9.16,14.2,0.06,58.0,15.3,84.0,7.6,6.0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
3,305.96,1.04,1.02,66,292.65,6.19,14.2,0.06,58.0,15.3,84.0,7.6,6.0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True
4,320.76,1.04,1.02,100,293.00,17.93,14.2,0.06,58.0,15.3,84.0,7.6,6.0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,True


In [31]:
#find where the accurateley predicted trues are
dfpred = dfResult.loc[dfResult['FIRE_DETECTED']==True]
dfpred = dfpred.loc[dfpred['PredResult']==True]
dfpred

,brightness,scan,track,confidence,bright_t31,frp,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,1,Aqua,Terra,MODIS,Arkansas-White-Red Region,California Region,Great Basin Region,Great Lakes Region,Hawaii Region,Lower Colorado Region,Lower Mississippi Region,Mid Atlantic Region,Missouri Region,New England Region,Ohio Region,Pacific Northwest Region,Rio Grande Region,Souris-Red-Rainy Region,South Atlantic-Gulf Region,Tennessee Region,Texas-Gulf Region,Upper Colorado Region,Upper Mississippi Region,FIRE_DETECTED,PredResult
5064,326.38,1.07,1.03,82,298.86,21.35,1.8,0.00,79.0,10.2,48.0,11.6,116.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,True,True
5065,309.06,1.06,1.03,40,297.31,4.72,1.8,0.00,79.0,10.2,48.0,11.6,116.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,True,True
29277,353.18,0.48,0.48,50,306.15,16.63,1.6,0.00,63.0,13.0,34.0,14.4,383.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
29638,326.07,0.38,0.36,50,280.56,2.71,1.6,0.00,47.0,12.0,37.0,13.3,18.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
29639,308.74,0.38,0.36,50,279.51,1.09,1.6,0.00,47.0,12.0,37.0,13.3,18.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
29915,332.31,0.56,0.43,50,300.46,7.11,7.1,0.00,57.0,28.9,15.0,12.1,52.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
30124,345.62,0.42,0.37,50,288.39,7.97,1.6,0.00,47.0,12.0,37.0,13.3,24.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
30125,338.73,0.42,0.37,50,285.24,7.97,1.6,0.00,47.0,12.0,37.0,13.3,24.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
30322,300.18,0.43,0.38,50,281.51,0.61,24.4,0.00,54.0,28.5,91.0,10.7,34.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True
30723,334.17,0.47,0.40,50,300.13,181.74,11.9,0.02,48.0,12.5,46.0,14.6,107.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,True


In [32]:
#get a full data frame prepared for merging
dfFullMerge = dfFULL.filter(['brightness', 'scan', 'track', 'bright_t31', 'frp'], axis=1)
dfFullMerge

,brightness,scan,track,bright_t31,frp
0,304.22,1.04,1.02,292.23,4.15
1,313.08,1.04,1.02,292.95,11.20
2,310.56,1.04,1.02,292.60,9.16
3,305.96,1.04,1.02,292.65,6.19
4,320.76,1.04,1.02,293.00,17.93
...,...,...,...,...,...
232613,367.00,0.63,0.72,299.12,50.76
232614,367.00,0.63,0.72,303.01,20.46
232615,367.00,0.63,0.72,300.61,21.17
232616,339.91,0.63,0.72,297.45,20.46


In [33]:
#prepare 
dfpredMerge = dfpred.filter(['brightness', 'scan', 'track', 'bright_t31', 'frp'], axis=1)
dfpredMerge

,brightness,scan,track,bright_t31,frp
5064,326.38,1.07,1.03,298.86,21.35
5065,309.06,1.06,1.03,297.31,4.72
29277,353.18,0.48,0.48,306.15,16.63
29638,326.07,0.38,0.36,280.56,2.71
29639,308.74,0.38,0.36,279.51,1.09
29915,332.31,0.56,0.43,300.46,7.11
30124,345.62,0.42,0.37,288.39,7.97
30125,338.73,0.42,0.37,285.24,7.97
30322,300.18,0.43,0.38,281.51,0.61
30723,334.17,0.47,0.40,300.13,181.74


In [37]:
DetectedList=[] #gets a list of detections
for index, row in dfpredMerge.iterrows():
    dfFound=dfFULL[(dfFULL['brightness']==row['brightness'])&(dfFULL['scan']==row['scan'])&(dfFULL['track']==row['track'])&(dfFULL['bright_t31']==row['bright_t31'])&(dfFULL['frp']==row['frp'])]
    DetectedList.append(dfFound)

In [38]:
DetectedList

[      latitude  longitude  brightness  scan  track    acq_date  acq_time  \
 5064  34.22024  -87.23557      326.38  1.07   1.03  2022-04-23      1639   
 
      satellite instrument confidence version  bright_t31    frp daynight  \
 5064     Terra      MODIS         82  6.1NRT      298.86  21.35        D   
 
        lat  long  hour               date_loc             date_hour_loc  \
 5064  34.2 -87.2  16.0  2022-04-23,34.2,-87.2  2022-04-23,16,34.2,-87.2   
 
          newdate nearestStation  StationDist        Date  Station Id  \
 5064  2022-04-23     Kingsville    70.801616  2022-04-23      2206.0   
 
      State Code Network Code Station Name  Elevation (ft)  Latitude  \
 5064         TX         SCAN   Kingsville            67.0  27.54945   
 
       Longitude County Name          HUC2 Name  \
 5064  -97.88223     Kleberg  Texas-Gulf Region   
 
                               HUC4 Name                   HUC6 Name  \
 5064  Nueces-Southwestern Texas Coastal  Southwestern Texas Coa

Here are the actual detected incidents from the list above.

- BK Inman Creek A
- Beaver River Fire
- Sand Creek
- WILDFLOWER
- County Road 56
- Clear Creek
- Persimmon Gully
- HOMOCHITTO BB25
- LAC-122039
- Sheridan Lake
- Wolf Swamp
- C-24 28
- BIGGS
- MIDDLE MARCH

In [39]:
len(DetectedList) 

48

In [43]:
#take a look at the WFIGS dataframe to see what was predicted accuratley 
incidentnames=[
    'BK Inman Creek A',
    'Beaver River Fire',
    'Sand Creek',
    'WILDFLOWER',
    'County Road 56',
    'Clear Creek',
    'Persimmon Gully',
    'HOMOCHITTO BB25',
    'LAC-122039,'
    'Sheridan Lake,'
    'Wolf Swamp,'
    'C-24 28,'
    'BIGGS',
    'MIDDLE MARCH'
]
dfIncidents = dfFULL[dfFULL['IncidentName'].isin(incidentnames)]
dfIncidents

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),date_new,nearbydetections,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc,FIRE_DETECTED
1093,35.39983,-100.70329,317.49,3.85,1.83,2022-04-06,353,Terra,MODIS,95,6.1NRT,282.73,122.79,N,35.4,-100.7,3.0,"2022-04-06,35.4,-100.7","2022-04-06,03,35.4,-100.7",2022-04-06,Schell-Osage,57.456680,2022-04-06,2193.0,MO,SCAN,Schell-Osage,793.0,37.98720,-94.03640,St. Clair,Missouri Region,Gasconade-Osage,Osage,Harry S. Truman Reservoir,12.8,0.21,0.66,52.0,31.3,36.6,52.0,0.5,2022-04-06,12.0,-100.688722,35.430707,248293.0,NaN,DEFAULT,NaN,2022/04/07 23:47:54+00,2022/04/11 00:59:59+00,4480.0,3200.0,TXTIC,1.0,NaN,NaN,NaN,Minimal,NaN,Smoldering,NaN,Human,Equipment and vehicle use,Electric motor/power tools/battery,NaN,NaN,2022/04/06 03:24:51+00,Type 5 Incident,2022/04/11 01:01:11+00,0.0,100.0,0.0,0.0,NaN,NaN,SACC,2022/04/09 00:45:00+00,2022/04/08 12:59:59+00,2022/04/09 00:45:00+00,F,Type 4 IC,Sand Creek,"7 miles East of Lefors, TX",WF,FI,35.430701,-100.688713,NaN,NaN,{EBFBF700-B48B-4354-AAE2-5AF9532B91DE},NaN,0.0,NaN,NaN,NaN,NaN,0.0,222372,100.0,100.0,Lefors,Gray,TXTIC,48179,State,TXTXS,NaN,Private,Private,NaN,NaN,NaN,NaN,NaN,NaN,SA01,SFS,TXTXS,US-TX,NaN,NaN,Short Grass (1 foot),Tall Grass (2.5 feet),33.0,2022-TXTXS-222372,No Decision,wires,wires,0.0,NaN,NaN,2022/04/06 06:13:49+00,2022/04/11 16:15:30+00,IRWIN,{51652647-E0DA-4498-ADDC-AD4F24A24F69},0.0,NaN,NaN,2022-04-06,35.4,-100.7,"2022-04-06,35.4,-100.7",True
1094,35.41556,-100.70926,305.08,3.85,1.83,2022-04-06,353,Terra,MODIS,63,6.1NRT,282.13,61.72,N,35.4,-100.7,3.0,"2022-04-06,35.4,-100.7","20

As mentioned in the intro, the fires Detected are Actually responded to Fires. If you google the incidents you will find results on several. Although the model had a not great recall, it did predict several wildfire starts accuiratley between May 1, 2022 and June 8, 2022. Below are several links of these wildfire which the model accuratley predicted the start of.

Beaver River Fire: https://abc7amarillo.com/news/local/beave-river-fire-2-firefighters-injured-2-homes-destroyed-24500-acres-burned 

County Road 56: https://thenwfireblog.com/2022/04/12/co-wildfire-county-road-56-fire-1/ 

Persimmon Gully: https://www.kplctv.com/2022/04/08/bud-bennett-fire-remains-within-containment-lines/ 

C-24 28: https://inciweb.nwcg.gov/incident/8069/ 

BIGGS: https://www.fireweatheravalanche.org/wildfire/incident/247104/new-mexico/biggs-fire 

MIDDLE MARCH: https://www.youtube.com/watch?v=krl5LZAIFw8 

https://www.myheraldreview.com/news/cochise_county/authorities-battling-middle-march-fire-in-dragoon-mountains/article_001567a0-b9d6-11ec-aaf4-77c7db6ba7ee.html


